In [11]:
# Import packages
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import bs4
from pprint import pprint

In [12]:
# Get team ID based on team name
def get_team_id(team):
  for team in teams:
    if team['teamName'] == team:
      return team['teamId']
  
# Get player ID based on player name
def get_player_id(first, last):
  for player in players:
    if player['firstName'] == first and player['lastName'] == last:
      return player['playerId']
  

In [29]:
import requests
teams = requests.get(
    "https://www.espn.com/mens-college-basketball/team/roster/_/id/194")
resp

<Response [200]>

In [30]:
soup = bs4.BeautifulSoup(teams.text,"html.parser")

In [31]:
pprint (soup)


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<!-- ESPNFITT | 31b9da5d3672 | 5705 | 0efe2a577e3e5c8d83309f9087b9b9075ea04150 | Mon, 09 May 2022 22:56:57 GMT -->
<script async="" src="https://dcf.espn.com/TWDC-DTCI/prod/Bootstrap.js" type="text/javascript"></script>
<script type="text/javascript">
        ;(function(){
            function gc(n){n=document.cookie.match("(^|;) ?"+n+"=([^;]*)(;|$)");return n?n[2]:null}function sc(n){document.cookie=n}function smpl(n){n/=100;return!!n&&Math.random()<=n}var _nr=!1,_nrCookie=gc("_nr");null!==_nrCookie?"1"===_nrCookie&&(_nr=!0):smpl(100)?(_nr=!0,sc("_nr=1; path=/")):(_nr=!1,sc("_nr=0; path=/"));;
            _nr && window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true}};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)re

In [32]:
t=soup.find("table")

In [33]:
 links = t.find_all("a")
 pprint(links)

[<a class="AnchorLink clr-gray-04" href="/mens-college-basketball/team/roster/_/id/194/sort/position/dir/asc" tabindex="0">POS</a>,
 <a class="AnchorLink clr-gray-04" href="/mens-college-basketball/team/roster/_/id/194/sort/height/dir/asc" tabindex="0">HT</a>,
 <a class="AnchorLink clr-gray-04" href="/mens-college-basketball/team/roster/_/id/194/sort/weight/dir/asc" tabindex="0">WT</a>,
 <a class="AnchorLink clr-gray-04" href="/mens-college-basketball/team/roster/_/id/194/sort/experience/dir/asc" tabindex="0">Class</a>,
 <a class="AnchorLink" href="https://www.espn.com/mens-college-basketball/player/_/id/4397236/justin-ahrens" tabindex="0"><div class="headshot inline-block relative TableHeadshot roster-headshot headshot--sm athlete silo"><figure class="Image aspect-ratio--parent"><div class="RatioFrame aspect-ratio--1x1"></div><div class="Image__Wrapper aspect-ratio--child"></div></figure></div></a>,
 <a class="AnchorLink" href="https://www.espn.com/mens-college-basketball/player/_/id/

In [34]:
hrefs =[]
names = []
player_ids =[]
for link in links:
    
    hrefs.append(link["href"])
    names.append(link.text)
    player_ids.append(link["data-player-uid"].split(":")[3])
print(hrefs)
print(names)
print(player_ids)

KeyError: 'data-player-uid'

In [97]:
df=pd.DataFrame(
{
    "link":hrefs,
    "name":names,
    "id":player_ids,
})
df

,link,name,id
0,https://www.espn.com/mens-college-basketball/p...,E.J. Liddell,4432821
1,https://www.espn.com/mens-college-basketball/p...,Malaki Branham,4565201
2,https://www.espn.com/mens-college-basketball/p...,Kyle Young,4277974
3,https://www.espn.com/mens-college-basketball/p...,Zed Key,4432401
4,https://www.espn.com/mens-college-basketball/p...,Jamari Wheeler,4277978
5,https://www.espn.com/mens-college-basketball/p...,Justice Sueing,4278141
6,https://www.espn.com/mens-college-basketball/p...,Justin Ahrens,4397236
7,https://www.espn.com/mens-college-basketball/p...,Meechie Johnson Jr.,4710770
8,https://www.espn.com/mens-college-basketball/p...,Cedric Russell,4280257
9,https://www.espn.com/mens-college-basketball/p...,Eugene Brown III,4433179


In [5]:
def get_player_shotchartdetail(player_name, season_id):
    """
    Parameters
    ----------
    player_name: name of the player with Capital
    season_id: ex. 2021-22
    """
    
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    print (nba_players)
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

In [7]:
%matplotlib inline

In [8]:
##Import pandas
import pandas as pd
## Read in Data to a Pandas Data Frame
unc_shot_df = pd.read_csv('/content/unc-shot-data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/unc-shot-data.csv'

In [9]:
### IMPORTS ###
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
## Plot
cmap = plt.cm.Blues_r
joint_shot_chart = sns.jointplot(unc_shot_df['event_coord_y'], unc_shot_df['event_coord_x'], stat_func=None, kind='kde', space=0, color=cmap(0.1), cmap=cmap, n_levels=50)
joint_shot_chart.fig.set_size_inches(12,11)
ax = joint_shot_chart.ax_joint
draw_court(ax,color='white')
# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom='off', labelleft='off')
# Add Data Scource
ax.text(600,600,'Data Source: NCAA Basketball/Google Cloud',fontsize=12)
# Descending values along the axis from left to right
ax.set_xlim(600,0)
ax.set_ylim(564,0)
plt.show()

NameError: name 'unc_shot_df' is not defined

In [53]:
import requests
headers = {'User-Agent': '<your user agent>'}
r = requests.get(base_url, params=params, headers=headers)

NameError: name 'base_url' is not defined

In [54]:
# Data and NBA API
import pandas as pd
import numpy as np

from scipy.stats import norm, gaussian_kde, percentileofscore

pd.options.display.max_columns = None
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

# For Shot Chart
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import cm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch

sns.set_style('white')
sns.set_color_codes()

In [55]:
json_data = r.json()
result = json_data['resultSets'][0]import pandas as pd
columns = result['headers']
data = result['rowSet']
df_shots = pd.DataFrame(data=data, columns=columns)
df_shots.head()

SyntaxError: invalid syntax (3377012286.py, line 2)

In [56]:
player_shotchart_df, league_avg = get_player_shotchartdetail('Chet Holmgren', '2021-22')

NameError: name 'get_player_shotchartdetail' is not defined

In [57]:
player_shotchart_df

NameError: name 'player_shotchart_df' is not defined

In [58]:
player_shotchart_df['LOC_X'].isnull().any().any()

NameError: name 'player_shotchart_df' is not defined

In [59]:
league_avg

NameError: name 'league_avg' is not defined

In [60]:
def draw_court(ax=None, color="blue", lw=1, shotzone=False, outer_lines=False):
    """Returns an axes with a basketball court drawn onto to it.
    This function draws a court based on the x and y-axis values that the NBA
    stats API provides for the shot chart data.  For example the center of the
    hoop is located at the (0,0) coordinate.  Twenty-two feet from the left of
    the center of the hoop in is represented by the (-220,0) coordinates.
    So one foot equals +/-10 units on the x and y-axis.
    Parameters
    ----------
    ax : Axes, optional
        The Axes object to plot the court onto.
    color : matplotlib color, optional
        The color of the court lines.
    lw : float, optional
        The linewidth the of the court lines.
    outer_lines : boolean, optional
        If `True` it draws the out of bound lines in same style as the rest of
        the court.
    Returns
    -------
    ax : Axes
        The Axes object with the court on it.
    """
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -12.5), 60, 0, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the right side 3pt lines, it's 14ft long before it arcs
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    # Create the right side 3pt lines, it's 14ft long before it arcs
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    
    # Draw shotzone Lines
    # Based on Advanced Zone Mode
    if (shotzone == True):
        inner_circle = Circle((0, 0), radius=80, linewidth=lw, color='black', fill=False)
        outer_circle = Circle((0, 0), radius=160, linewidth=lw, color='black', fill=False)
        corner_three_a_x =  Rectangle((-250, 92.5), 30, 0, linewidth=lw, color=color)
        corner_three_b_x = Rectangle((220, 92.5), 30, 0, linewidth=lw, color=color)
        
        # 60 degrees
        inner_line_1 = Rectangle((40, 69.28), 80, 0, 60, linewidth=lw, color=color)
        # 120 degrees
        inner_line_2 = Rectangle((-40, 69.28), 80, 0, 120, linewidth=lw, color=color)
        
        # Assume x distance is also 40 for the endpoint
        inner_line_3 = Rectangle((53.20, 150.89), 290, 0, 70.53, linewidth=lw, color=color)
        inner_line_4 = Rectangle((-53.20, 150.89), 290, 0, 109.47, linewidth=lw, color=color)
        
        # Assume y distance is also 92.5 for the endpoint
        inner_line_5 = Rectangle((130.54, 92.5), 80, 0, 35.32, linewidth=lw, color=color)
        inner_line_6 = Rectangle((-130.54, 92.5), 80, 0, 144.68, linewidth=lw, color=color)
        
        
        # List of the court elements to be plotted onto the axes
        court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                          bottom_free_throw, restricted, corner_three_a,
                          corner_three_b, three_arc, center_outer_arc,
                          center_inner_arc, inner_circle, outer_circle,
                          corner_three_a_x, corner_three_b_x,
                          inner_line_1, inner_line_2, inner_line_3, inner_line_4, inner_line_5, inner_line_6]
    else:
        # List of the court elements to be plotted onto the axes
        court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                          bottom_free_throw, restricted, corner_three_a,
                          corner_three_b, three_arc, center_outer_arc,
                          center_inner_arc]
    
    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)
        

    return ax

In [61]:
def shot_chart(data, title="", color="b",
               xlim=(-250, 250), ylim=(422.5, -47.5), line_color="blue",
               court_color="white", court_lw=2, outer_lines=False,
               flip_court=False, gridsize=None, 
               ax=None, despine=False, **kwargs):

    if ax is None:
        ax = plt.gca()

    if not flip_court:
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
    else:
        ax.set_xlim(xlim[::-1])
        ax.set_ylim(ylim[::-1])

    ax.tick_params(labelbottom="off", labelleft="off")
    ax.set_title(title, fontsize=18)

    # draws the court
    draw_court(ax, color=line_color, lw=court_lw, outer_lines=outer_lines)

    # separate color by make or miss    
    x_missed = data[data['EVENT_TYPE'] == 'Missed Shot']['LOC_X']
    y_missed = data[data['EVENT_TYPE'] == 'Missed Shot']['LOC_Y']

    x_made = data[data['EVENT_TYPE'] == 'Made Shot']['LOC_X']
    y_made = data[data['EVENT_TYPE'] == 'Made Shot']['LOC_Y']

    # plot missed shots
    ax.scatter(x_missed, y_missed, c='r', marker="x", s=300, linewidths=3, **kwargs)
    # plot made shots
    ax.scatter(x_made, y_made, facecolors='none', edgecolors='g', marker="o", s=100, linewidths=3, **kwargs)
        
    # Set the spines to match the rest of court lines, makes outer_lines
    # somewhate unnecessary
    for spine in ax.spines:
        ax.spines[spine].set_lw(court_lw)
        ax.spines[spine].set_color(line_color)

    if despine:
        ax.spines["top"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)

    return ax

In [62]:
# Set the size for our plots
plt.rcParams['figure.figsize'] = (12, 11)

In [66]:
library(rjson)
# shot data for Stephen Curry
playerID <- 201939
shotURL <- paste("http://stats.nba.com/stats/shotchartdetail?CFID=33&CFPARAMS=2014-15&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerID=",playerID,"&PlayerPosition=&PlusMinus=N&Position=&Rank=N&RookieYear=&Season=2014-15&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision=&mode=Advanced&showDetails=0&showShots=1&showZones=0", sep = "")
# import from JSON
shotData <- fromJSON(file = shotURL, method="C")

NameError: name 'playerID' is not defined

In [67]:
import requests
resp = requests.get(
    "http://stats.nba.com/stats/shotchartdetail?CFID=33&CFPARAMS=2014-15&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerID=",playerID,"&PlayerPosition=&PlusMinus=N&Position=&Rank=N&RookieYear=&Season=2014-15&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision=&mode=Advanced&showDetails=0&showShots=1&showZones=0")
resp

NameError: name 'playerID' is not defined

In [63]:
shot_chart(player_shotchart_df, title="KD Shotting Chart")
plt.show()
print("FG%: ", "{0:.3f}".format(player_shotchart_df['SHOT_MADE_FLAG'].sum()/len(player_shotchart_df)), "({0}-{1})".format(player_shotchart_df['SHOT_MADE_FLAG'].sum(),len(player_shotchart_df)))

NameError: name 'player_shotchart_df' is not defined